<a href="https://colab.research.google.com/github/rubanzasilva/mnist_basics/blob/main/gold_futures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#This imports and sets up everything you will need for this notebook
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [ ]:
#hide
from fastbook import *
from fastai.tabular.all import *
import torch, numpy as np, pandas as pd

matplotlib.rc('image', cmap='Greys')
np.set_printoptions(linewidth=140)
torch.set_printoptions(linewidth=140, sci_mode=False, edgeitems=7)
pd.set_option('display.width', 140)


### Access Drive

I have stored my gold futures and usd currency historical datasets ranging from 2nd February 2019 till 7th March 2024 in my google drive.

To access these i will mount google drive with the codecell below which gives me access to my google drive files.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


I navigate and use shell code to navigate to my gold_futures folder stored in google drive.

In [ ]:
!ls drive/MyDrive/Rubanza/datasets/gold_futures

gold_futures_historical_data.csv  usd_historical_data.csv


In [ ]:
!ls drive/MyDrive/Rubanza/datasets/gold_futures/x/

gold.csv  usd.csv


In [ ]:
path = Path('drive/MyDrive/Rubanza/datasets/gold_futures')
path


Path('drive/MyDrive/Rubanza/datasets/gold_futures')

### Look at our data

In [ ]:
gold_features_df = pd.read_csv(path/'gold_futures_historical_data.csv')
usd_currency_df = pd.read_csv(path/'usd_historical_data.csv')


In [ ]:
gold_features_df.shape,usd_currency_df.shape

((1327, 7), (1326, 7))

The gold_features seems to have one more column than usd_currency

In [ ]:
usd_currency_df.shape

(1326, 7)

In [ ]:
gold_features_df.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,03/07/2024,"2,163.75","2,154.90","2,168.10","2,151.65",NaN,0.26%
1,03/06/2024,"2,158.20","2,136.40","2,160.70","2,131.90",319.76K,0.76%
2,03/05/2024,"2,141.90","2,123.30","2,150.50","2,118.50",283.15K,0.73%
3,03/04/2024,"2,126.30","2,091.60","2,128.40","2,088.10",328.25K,1.46%
4,03/01/2024,"2,095.70","2,052.80","2,097.10","2,047.00",330.59K,2.00%


In [ ]:
usd_currency_df.head()

,Date,Price,Open,High,Low,Vol.,Change %
0,03/07/2024,95.266,95.30,95.300,95.266,NaN,0.41%
1,03/06/2024,94.880,95.59,96.075,94.550,NaN,-0.39%
2,03/05/2024,95.250,95.67,96.075,94.700,NaN,-0.01%
3,03/04/2024,95.260,95.60,96.280,94.825,NaN,-0.16%
4,03/01/2024,95.410,96.16,96.450,95.725,NaN,-0.36%


In [ ]:
gold_features_df

,Date,Price,Open,High,Low,Vol.,Change %
0,03/07/2024,"2,163.75","2,154.90","2,168.10","2,151.65",NaN,0.26%
1,03/06/2024,"2,158.20","2,136.40","2,160.70","2,131.90",319.76K,0.76%
2,03/05/2024,"2,141.90","2,123.30","2,150.50","2,118.50",283.15K,0.73%
3,03/04/2024,"2,126.30","2,091.60","2,128.40","2,088.10",328.25K,1.46%
4,03/01/2024,"2,095.70","2,052.80","2,097.10","2,047.00",330.59K,2.00%
...,...,...,...,...,...,...,...
1322,02/08/2019,"1,318.50","1,313.20","1,319.50","1,311.50",150.61K,0.33%
1323,02/07/2019,"1,314.20","1,308.10","1,315.80","1,306.40",166.76K,-0.02%
1324,02/06/2019,"1,314.40","1,317.60","1,319.70","1,309.60",137.25K,-0.36%
1325,02/05/2019,"1,319.20","1,317.10","1,321.00","1,314.80",129.01K,-0.01%


In [ ]:
usd_currency_df

,Date,Price,Open,High,Low,Vol.,Change %
0,03/07/2024,95.266,95.300,95.300,95.266,NaN,0.41%
1,03/06/2024,94.880,95.590,96.075,94.550,NaN,-0.39%
2,03/05/2024,95.250,95.670,96.075,94.700,NaN,-0.01%
3,03/04/2024,95.260,95.600,96.280,94.825,NaN,-0.16%
4,03/01/2024,95.410,96.160,96.450,95.725,NaN,-0.36%
...,...,...,...,...,...,...,...
1321,02/08/2019,108.730,109.600,109.905,109.145,19.92K,-0.40%
1322,02/07/2019,109.170,109.175,109.900,109.150,16.46K,0.26%
1323,02/06/2019,108.890,109.000,109.580,108.820,17.65K,0.08%
1324,02/05/2019,108.800,108.965,109.430,108.570,19.82K,0.10%


The dataset seem to be missing rows for some dates.From intuition am guessing these are probably weekends, holidays etc., days on which the markets might be closed.

But as long as both datasets have the same number of rows, they might all be missing the same exact dates.so we check the shapes which reveal that gold_features seems to have one more row than usd_currency.

Given that both datasets are supposed to contain data from 4th November 2019 to 7th March 2024, the extra row in gold_features_df could be due to:

Duplicate entries for certain dates that exist in both datasets.
Missing data for some dates in one dataset compared to the other.
A date that exists in one dataset but is not correctly aligned with the other dataset's dates.


These files both have the same column names so is it possible to merge them ?
We can try to solve this by renaming some of the columns in the usd dataset.

In [ ]:
usd_currency_df = usd_currency_df.rename(columns={'Price': 'USD_Price', 'Open': 'USD_Open', 'High': 'USD_High', 'Low': 'USD_Low', 'Vol.': 'USD_Vol', 'Change %': 'USD_Change'})

In [ ]:
usd_currency_df

,Date,USD_Price,USD_Open,USD_High,USD_Low,USD_Vol,USD_Change
0,03/07/2024,95.266,95.300,95.300,95.266,NaN,0.41%
1,03/06/2024,94.880,95.590,96.075,94.550,NaN,-0.39%
2,03/05/2024,95.250,95.670,96.075,94.700,NaN,-0.01%
3,03/04/2024,95.260,95.600,96.280,94.825,NaN,-0.16%
4,03/01/2024,95.410,96.160,96.450,95.725,NaN,-0.36%
...,...,...,...,...,...,...,...
1321,02/08/2019,108.730,109.600,109.905,109.145,19.92K,-0.40%
1322,02/07/2019,109.170,109.175,109.900,109.150,16.46K,0.26%
1323,02/06/2019,108.890,109.000,109.580,108.820,17.65K,0.08%
1324,02/05/2019,108.800,108.965,109.430,108.570,19.82K,0.10%


In [ ]:
usd_currency_df['USD_Price'].dtype

dtype('float64')

In [ ]:
usd_currency_df['Date'].dtype

dtype('O')

### Date Formating

Ensure the date is in a consistent format.We can do this by converting the dates from strings to datetime objects using pd.datetime

In [ ]:
# Ensure the 'Date' column is in a consistent format
usd_currency_df['Date'] = pd.to_datetime(usd_currency_df['Date'])
gold_features_df['Date'] = pd.to_datetime(gold_features_df['Date'])

To simplify things we can use the FastAi's make_date instead of pd.datetime which is a more general function.

In [ ]:
#make_date(gold_features_df,'Date')

In [ ]:
#make_date(usd_currency_df,'Date')

In [ ]:
gold_features_df

,Date,Price,Open,High,Low,Vol.,Change %
0,2024-03-07,"2,163.75","2,154.90","2,168.10","2,151.65",NaN,0.26%
1,2024-03-06,"2,158.20","2,136.40","2,160.70","2,131.90",319.76K,0.76%
2,2024-03-05,"2,141.90","2,123.30","2,150.50","2,118.50",283.15K,0.73%
3,2024-03-04,"2,126.30","2,091.60","2,128.40","2,088.10",328.25K,1.46%
4,2024-03-01,"2,095.70","2,052.80","2,097.10","2,047.00",330.59K,2.00%
...,...,...,...,...,...,...,...
1322,2019-02-08,"1,318.50","1,313.20","1,319.50","1,311.50",150.61K,0.33%
1323,2019-02-07,"1,314.20","1,308.10","1,315.80","1,306.40",166.76K,-0.02%
1324,2019-02-06,"1,314.40","1,317.60","1,319.70","1,309.60",137.25K,-0.36%
1325,2019-02-05,"1,319.20","1,317.10","1,321.00","1,314.80",129.01K,-0.01%


In [ ]:
usd_currency_df

,Date,USD_Price,USD_Open,USD_High,USD_Low,USD_Vol,USD_Change
0,2024-03-07,95.266,95.300,95.300,95.266,NaN,0.41%
1,2024-03-06,94.880,95.590,96.075,94.550,NaN,-0.39%
2,2024-03-05,95.250,95.670,96.075,94.700,NaN,-0.01%
3,2024-03-04,95.260,95.600,96.280,94.825,NaN,-0.16%
4,2024-03-01,95.410,96.160,96.450,95.725,NaN,-0.36%
...,...,...,...,...,...,...,...
1321,2019-02-08,108.730,109.600,109.905,109.145,19.92K,-0.40%
1322,2019-02-07,109.170,109.175,109.900,109.150,16.46K,0.26%
1323,2019-02-06,108.890,109.000,109.580,108.820,17.65K,0.08%
1324,2019-02-05,108.800,108.965,109.430,108.570,19.82K,0.10%


### Handle missing dates

First let me confirm there are no repeated dates/duplicates




In [ ]:
gold_features_duplicates = gold_features_df.duplicated(subset=['Date'], keep=False) # Check for duplicates based on 'Date' column

# Print the number of duplicate rows
print(f"Number of duplicate rows in gold features is: {gold_features_duplicates.sum()}")

Number of duplicate rows in gold features is: 0


In [ ]:
usd_currency_duplicates = usd_currency_df.duplicated(subset=['Date'], keep=False) # Check for duplicates based on 'Date' column

# Print the number of duplicate rows
print(f"Number of duplicate rows in usd_currency_df is: {usd_currency_duplicates.sum()}")

Number of duplicate rows in usd_currency_df is: 0


Use datepart and add_cyclic_datepart


search for how fastai handles dates , or any of the date formating methods/functions.

Add more features with datepart

### Create Data Dictionary

### Further Data Transformations

We also have to deal with our common data issues such as missing values, long tail distributions etc.
We shall do this before we merge the datasets in this case.

To check out the potential issues in our data we use df.describe which returns a summary for our dataset.

We get a summary for our numeric columns first.

In [42]:
import pandas as pd

In [ ]:
gold_features_df.describe(include=(np.number))

Our dataset seems to have some NaN values so we check which columns have Nan values and get their count.

In [40]:
gold_features_df.isna().sum()

Date         0
Price        0
Open         0
High         0
Low          0
Vol.        22
Change %     0
dtype: int64

### Merge Datasets

Merge both datasets using pd.merge

In [ ]:
# Merge the DataFrames on the 'Date' column
#train_df = pd.merge(usd_currency_df, gold_features_df, on='Date', how='inner')